# ETL to DWH - mkoe

In [1]:
import os
import ast
import json
import pandas as pd
import sqlite3

## Load
### Einlesen der CSV

In [2]:
CSV_FILE_PATH = os.path.join("..", "input", "movies_metadata.csv")
SQLITE_FILE_PATH = os.path.join('..', "output", "dwh.sqlite3")

In [3]:
movies_df = pd.read_csv(CSV_FILE_PATH)

C:\Users\mkoenig\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## Transform

### Drei falsche Datensätze löschne

In [5]:
#Löschen aller nicht integer-Spalten
movies_df = movies_df.drop(movies_df.loc[~movies_df["id"].str.isdigit()].index)

In [6]:
#Auslesen, ob ein Wert in der Spalte ids nicht ein integer ist
movies_df.loc[~movies_df["id"].str.isdigit()].index

Int64Index([], dtype='int64')

### Datentypen anschauen

#### Spalten in Movies löschen

In [7]:
movies_final_df = movies_df.drop(columns=["belongs_to_collection","spoken_languages","genres", "production_countries", "production_companies"])

In [8]:
movies_final_df.head()

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


#### Budget zu zahlen

In [9]:
movies_final_df['budget'] = movies_final_df['budget'].astype(int)
movies_final_df['id'] = movies_final_df['id'].astype(int)
movies_final_df['popularity'] = movies_final_df['popularity'].astype(float)

In [10]:
movies_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45463 non-null  object 
 1   budget             45463 non-null  int32  
 2   homepage           7779 non-null   object 
 3   id                 45463 non-null  int32  
 4   imdb_id            45446 non-null  object 
 5   original_language  45452 non-null  object 
 6   original_title     45463 non-null  object 
 7   overview           44509 non-null  object 
 8   popularity         45460 non-null  float64
 9   poster_path        45077 non-null  object 
 10  release_date       45376 non-null  object 
 11  revenue            45460 non-null  float64
 12  runtime            45203 non-null  float64
 13  status             45379 non-null  object 
 14  tagline            20412 non-null  object 
 15  title              45460 non-null  object 
 16  video              454


### Sichere Codeprüfung mit Eval-Funktionen

In [11]:
x= movies_df["genres"].apply(ast.literal_eval)

In [12]:
x

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                           [{'id': 35, 'name': 'Comedy'}]
                               ...                        
45461    [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...
45462                        [{'id': 18, 'name': 'Drama'}]
45463    [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...
45464                                                   []
45465                                                   []
Name: genres, Length: 45463, dtype: object

In [13]:
x[0][0]["id"]

16

In [14]:
movies_df.shape

(45463, 24)

In [15]:
s2 = "[{'id': print('hi'), 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [16]:
eval(s2)

hi


[{'id': None, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [17]:
#ast.literal_eval(s2)

## Genres auflösen in extra Tabelle

### Versuch in der Gruppe

Extraktion der Genres aus dem movies_df. Problem
Hierbei in der Spalte genres sind mit einem Dictionary mehrere ids und Genres verknüpft.

In [18]:
# DF bilden mit Genres und Ids der Filme
movie_df = movies_df[["genres", "id"]]

In [19]:
movie_df

,genres,id
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357
4,"[{'id': 35, 'name': 'Comedy'}]",11862
...,...,...
45461,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050
45462,"[{'id': 18, 'name': 'Drama'}]",111109
45463,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758
45464,[],227506


In [20]:
movies_genres = []
genre_names = []

for index, row in movie_df.iterrows():
    #print (row["id"])
    genres = ast.literal_eval(row["genres"])
    for genre in genres:
        #print ("--> ",genre['id'])
        movie_genre = dict(
            movie_id=row["id"],
            genre_id=genre['id'],
        )
        movies_genres.append(movie_genre)
        genre_name = dict(
            genre_id=genre['id'],
            genre_name=genre['name']
        )
        genre_names.append(genre_name)

In [21]:
genre_names_df = pd.DataFrame(genre_names).drop_duplicates()

In [22]:
genre_names_df

,genre_id,genre_name
0,16,Animation
1,35,Comedy
2,10751,Family
3,12,Adventure
4,14,Fantasy
6,10749,Romance
9,18,Drama
12,28,Action
13,80,Crime
15,53,Thriller


In [23]:
movies_genre_df = pd.DataFrame(movies_genres)

In [24]:
movies_genre_df

,movie_id,genre_id
0,862,16
1,862,35
2,862,10751
3,8844,12
4,8844,14
...,...,...
91089,439050,10751
91090,111109,18
91091,67758,28
91092,67758,18


### Marcels Code

Extraktion der Genres aus dem movies_df. Problem Hierbei in der Spalte genres sind mit einem Dictionary mehrere ids und Genres verknüpft.

In [25]:
genres =[]
genres_movies = []

for i, m in movies_df.iterrows():
    mg = m["genres"]
    mg = ast.literal_eval(mg)
    for g in mg:
        genres.append(g)
        genres_movies.append(dict(
            movie_id=m["id"],
            genre_id=g["id"],
        ))



In [26]:
pd.DataFrame(genres).drop_duplicates()

,id,name
0,16,Animation
1,35,Comedy
2,10751,Family
3,12,Adventure
4,14,Fantasy
6,10749,Romance
9,18,Drama
12,28,Action
13,80,Crime
15,53,Thriller


In [27]:
pd.DataFrame(genres_movies)

,movie_id,genre_id
0,862,16
1,862,35
2,862,10751
3,8844,12
4,8844,14
...,...,...
91089,439050,10751
91090,111109,18
91091,67758,28
91092,67758,18


## Überführen in SQL

In [28]:
con = sqlite3.connect(SQLITE_FILE_PATH)

In [29]:
genre_names_df.to_sql("genres", con=con, if_exists="replace", index=False)

In [30]:
query = """
SELECT * FROM genres;
"""
pd.read_sql_query(query, con=con)

,genre_id,genre_name
0,16,Animation
1,35,Comedy
2,10751,Family
3,12,Adventure
4,14,Fantasy
5,10749,Romance
6,18,Drama
7,28,Action
8,80,Crime
9,53,Thriller


In [31]:
movies_genre_df.to_sql("movies_genre", con=con, if_exists="replace", index=False)

In [32]:
query = """
SELECT * FROM movies_genre;
"""
pd.read_sql_query(query, con=con)

,movie_id,genre_id
0,862,16
1,862,35
2,862,10751
3,8844,12
4,8844,14
...,...,...
91089,439050,10751
91090,111109,18
91091,67758,28
91092,67758,18


In [33]:
movies_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45463 non-null  object 
 1   budget             45463 non-null  int32  
 2   homepage           7779 non-null   object 
 3   id                 45463 non-null  int32  
 4   imdb_id            45446 non-null  object 
 5   original_language  45452 non-null  object 
 6   original_title     45463 non-null  object 
 7   overview           44509 non-null  object 
 8   popularity         45460 non-null  float64
 9   poster_path        45077 non-null  object 
 10  release_date       45376 non-null  object 
 11  revenue            45460 non-null  float64
 12  runtime            45203 non-null  float64
 13  status             45379 non-null  object 
 14  tagline            20412 non-null  object 
 15  title              45460 non-null  object 
 16  video              454

In [34]:
movies_final_df.to_sql("movies", con=con, if_exists="replace", index=False)

In [35]:
query = """
SELECT * FROM movies;
"""
pd.read_sql_query(query, con=con)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,None,Toy Story,0.0,7.7,5415.0
1,False,65000000,None,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,0.0,6.9,2413.0
2,False,0,None,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,0.0,6.5,92.0
3,False,16000000,None,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,0.0,6.1,34.0
4,False,0,None,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,0.0,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45458,False,0,http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,None,0.0,90.0,Released,Rising and falling between a man and woman,Subdue,0.0,4.0,1.0
45459,False,0,None,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,2011-11-17,0.0,360.0,Released,None,Century of Birthing,0.0,9.0,3.0
45460,False,0,None,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,2003-08-01,0.0,90.0,Released,A deadly game of wits.,Betrayal,0.0,3.8,6.0
45461,False,0,None,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,1917-10-21,0.0,87.0,Released,None,Satan Triumphant,0.0,0.0,0.0
